In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report
from pandas import get_dummies

In [76]:
dataset = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [77]:
a = pd.get_dummies(dataset['sex'], prefix = "sex")
b = pd.get_dummies(dataset['cp'], prefix = "cp")
c = pd.get_dummies(dataset['fbs'], prefix = "fbs")
d = pd.get_dummies(dataset['restecg'], prefix = "restecg")
e = pd.get_dummies(dataset['exang'], prefix = "exang")
f = pd.get_dummies(dataset['slope'], prefix = "slope")
g = pd.get_dummies(dataset['ca'], prefix = "ca")
h = pd.get_dummies(dataset['thal'], prefix = "thal")


frames = [dataset, a, b, c, d, e, f, g, h]


dataset2 = pd.concat(frames, axis = 1)


dataset2 = dataset2.drop(columns = ['sex','cp', 'fbs', 'restecg',
                                  'exang','slope','ca','thal'])

In [78]:
y = dataset2["target"]
X = dataset2.drop('target',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

In [79]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [80]:
lr = LogisticRegression()
model = lr.fit(X_train, y_train)
lr_predict = lr.predict(X_test)
lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_acc_score = accuracy_score(y_test, lr_predict)
print("confussion matrix")
print(lr_conf_matrix)
print("\n")
print("Accuracy of Logistic Regression:",lr_acc_score*100,'\n')
print(classification_report(y_test,lr_predict))

confussion matrix
[[ 84  14]
 [  6 101]]


Accuracy of Logistic Regression: 90.2439024390244 

              precision    recall  f1-score   support

           0       0.93      0.86      0.89        98
           1       0.88      0.94      0.91       107

    accuracy                           0.90       205
   macro avg       0.91      0.90      0.90       205
weighted avg       0.90      0.90      0.90       205



In [81]:
df2 = pd.read_excel('CTG.xls', sheet_name = 'Raw Data')
df2=df2.drop(["FileName","Date","SegFile","b","e","CLASS"],axis=1)
df2.head()

,LBE,LB,AC,FM,UC,ASTV,MSTV,ALTV,MLTV,DL,...,B,C,D,E,AD,DE,LD,FS,SUSP,NSP
0,120,120,0,0,0,73,0.5,43,2.4,0,...,0,0,0,0,0,0,0,1,0,2
1,132,132,4,0,4,17,2.1,0,10.4,2,...,0,0,0,0,1,0,0,0,0,1
2,133,133,2,0,5,16,2.1,0,13.4,2,...,0,0,0,0,1,0,0,0,0,1
3,134,134,2,0,6,16,2.4,0,23.0,2,...,0,0,0,0,1,0,0,0,0,1
4,132,132,4,0,5,16,2.4,0,19.9,0,...,1,0,0,0,0,0,0,0,0,1


In [82]:
y = df2["NSP"]
X = df2.drop("NSP",axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

In [83]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [84]:
lr = LogisticRegression()
model = lr.fit(X_train, y_train)
lr_predict = lr.predict(X_test)
lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_acc_score = accuracy_score(y_test, lr_predict)
print("confussion matrix")
print(lr_conf_matrix)
print("\n")
print("Accuracy of Logistic Regression:",lr_acc_score*100,'\n')
print(classification_report(y_test,lr_predict))

confussion matrix
[[322   4   0]
 [  5  53   0]
 [  0   0  42]]


Accuracy of Logistic Regression: 97.88732394366197 

              precision    recall  f1-score   support

           1       0.98      0.99      0.99       326
           2       0.93      0.91      0.92        58
           3       1.00      1.00      1.00        42

    accuracy                           0.98       426
   macro avg       0.97      0.97      0.97       426
weighted avg       0.98      0.98      0.98       426



In [86]:
DT = DecisionTreeClassifier(max_depth=5)
model = DT.fit(X_train, y_train)
lr_predict = DT.predict(X_test)
lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_acc_score = accuracy_score(y_test, lr_predict)
print("confussion matrix")
print(lr_conf_matrix)
print("\n")
print("Accuracy of Logistic Regression:",lr_acc_score*100,'\n')
print(classification_report(y_test,lr_predict))

confussion matrix
[[325   1   0]
 [  8  50   0]
 [  0   1  41]]


Accuracy of Logistic Regression: 97.65258215962442 

              precision    recall  f1-score   support

           1       0.98      1.00      0.99       326
           2       0.96      0.86      0.91        58
           3       1.00      0.98      0.99        42

    accuracy                           0.98       426
   macro avg       0.98      0.95      0.96       426
weighted avg       0.98      0.98      0.98       426

